# A featural model for relating multiple time series

In our application of interest, we are faced with a collection of N time series representing realizations of related dynamical phenomena.

### Per series dynamics
We model the dynamics of each time series as an autoregressive Hidden Markov Model (AR-HMM).<br>
<br>
<center> $z_t|z_{t-1} \sim \pi_{z_{t-1}}$ </center>
<br>
<center> $y_t = \sum_{l=1}^{r} A_{l,z_t}y_{t-l} + e_t(z_t) = \bf{A}_k \hat{y}_t + e_t(z_t)$ </center>
<br>
where $e_t(z_t) \sim \mathcal{N}(0, \Sigma_{z_t})$ and $\hat{y}_t = L^r(y_t)$ are the aggregated past observations.  We refer to $\bf{A}_k=[A_{1,k}, \dots, A_{r,k}]$ as the set of lag matrices.  Throughout, we denote the VAR parameters for the kth state as $\theta_k = \{A_k, \Sigma_k\}$ and refer to each VAR process as a dynamic behavior.


Given a previous set of mode-specific transition probabilities $\pi^{n-1}$, the global transition distribution $\beta^{(n-1})$, and dynamic parameters $\bf{\theta}^{(n-1)}$:

1. Set $\bf{\pi} = \bf{\pi}^{n-1}$, $\bf{\beta} = \bf{\beta}^{n-1}$, and $\bf{\theta} = \bf{\theta}^{n-1}$

In [ ]:
import numpy as np

def backwards_message_vec(likelihood,blockEnd,pi_z,pi_s):
    #Allocate storage space
    Kz=pi_z.shape[1]
    Ks=pi_s.shape[1]
    T=length(blockEnd)
    
    bwds_msg = np.ones(Kz,T)
    partial_marg = zeros(Kz,T)
    #Compute marginalized likelihoods for all times, integrating s_t
    if Kz==1 and Ks=1:
        marg_like =  squeeze(likelihood)
    else:
        marg_like = squeeze(np.sum(likelihood * pi_s[:,:,np.ones(1,1)]))

In [2]:
from __future__ import division
import numpy as np
from numpy.random import random
from numpy import newaxis as na
import scipy.stats as stats
import scipy.linalg

### Sampling functions

def sample_discrete(dist,size=[]):
    assert (dist >=0).all()
    cumvals = np.cumsum(dist)
    return np.sum(random(size)[...,na] * cumvals[-1] > cumvals, axis=-1)

def sample_niw(mu_0,lmbda_0,kappa_0,nu_0):
    '''
    Returns a sample from the normal/inverse-wishart distribution, conjugate
    prior for (simultaneously) unknown mean and unknown covariance in a
    Gaussian likelihood model. Returns covariance.  '''
    # this is completely copied from Matlab's implementation, ignoring
    # the copyright. I'm sorry.
    # reference: p. 87 in Gelman's Bayesian Data Analysis

    # first sample Sigma ~ IW(lmbda_0^-1,nu_0)
    lmbda = sample_invwishart(lmbda_0,nu_0) # lmbda = np.linalg.inv(sample_wishart(np.linalg.inv(lmbda_0),nu_0))
    # then sample mu | Lambda ~ N(mu_0, Lambda/kappa_0)
    mu = np.random.multivariate_normal(mu_0,lmbda / kappa_0)

    return mu, lmbda

def sample_invwishart(lmbda,dof):
    # TODO make a version that returns the cholesky
    # TODO allow passing in chol/cholinv of matrix parameter lmbda
    n = lmbda.shape[0]
    chol = np.linalg.cholesky(lmbda)

    if (dof <= 81+n) and (dof == np.round(dof)):
        x = np.random.randn(dof,n)
    else:
        x = np.diag(np.sqrt(stats.chi2.rvs(dof-(np.arange(n)))))
        x[np.triu_indices_from(x,1)] = np.random.randn(n*(n-1)/2)
    R = np.linalg.qr(x,'r')
    T = scipy.linalg.solve_triangular(R.T,chol.T).T
    return np.dot(T,T.T)

def sample_wishart(sigma, dof):
    '''
    Returns a sample from the Wishart distn, conjugate prior for precision matrices.
    '''

    n = sigma.shape[0]
    chol = np.linalg.cholesky(sigma)

    # use matlab's heuristic for choosing between the two different sampling schemes
    if (dof <= 81+n) and (dof == round(dof)):
        # direct
        X = np.dot(chol,np.random.normal(size=(n,dof)))
    else:
        A = np.diag(np.sqrt(np.random.chisquare(dof - np.arange(0,n),size=n)))
        A[np.tri(n,k=-1,dtype=bool)] = np.random.normal(size=(n*(n-1)/2.))
        X = np.dot(chol,A)

    return np.dot(X,X.T)

In [4]:
tmp={'a':1,'b':2}
print len(tmp)

2


(1, 10)